In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

# Parameters
VIX_THR = 15
START = "2010-01-01"
END = "2025-12-31"

# -----------------------------
# Download data
# -----------------------------
vix = yf.download("^VIX", start=START, end=END, progress=False)
upro = yf.download("UPRO", start=START, end=END, progress=False)

print("VIX rows:", len(vix))
print("UPRO rows:", len(upro))

# Check for errors
if vix.empty:
    raise ValueError("VIX data download failed — dataset is EMPTY.")
if upro.empty:
    raise ValueError("UPRO data download failed — dataset is EMPTY.")

# -----------------------------
# Merge using index to avoid scalar error
# -----------------------------
df = pd.merge(
    vix[["Close"]].rename(columns={"Close": "VIX"}),
    upro[["Close"]].rename(columns={"Close": "UPRO"}),
    left_index=True,
    right_index=True,
    how="inner"
)

df.dropna(inplace=True)

print(df.head())
print(df.tail())


/tmp/ipykernel_14249/4172319162.py:13: FutureWarning: YF.download() has changed argument auto_adjust default to True
  vix = yf.download("^VIX", start=START, end=END, progress=False)
/tmp/ipykernel_14249/4172319162.py:14: FutureWarning: YF.download() has changed argument auto_adjust default to True
  upro = yf.download("UPRO", start=START, end=END, progress=False)


VIX rows: 3998
UPRO rows: 3998
Price             VIX      UPRO
Ticker           ^VIX      UPRO
Date                           
2010-01-04  20.040001  2.077316
2010-01-05  19.350000  2.095168
2010-01-06  19.160000  2.101206
2010-01-07  19.059999  2.126804
2010-01-08  18.129999  2.147543
Price             VIX        UPRO
Ticker           ^VIX        UPRO
Date                             
2025-11-17  22.379999  108.910004
2025-11-18  24.690001  106.190002
2025-11-19  23.660000  107.320000
2025-11-20  26.420000  102.379997
2025-11-21  23.430000  105.379997


In [3]:
# -----------------------------
# Indicators: VIX MA5
# -----------------------------
df["VIX_MA5"] = df["VIX"].rolling(5).mean()

# -----------------------------
# Strategy rules
# -----------------------------
# Entry: VIX < thr AND VIX < MA5
enter = (df["VIX"] < VIX_THR) & (df["VIX"] < df["VIX_MA5"])

# Exit: VIX > thr AND VIX > MA5
exit_ = (df["VIX"] > VIX_THR) & (df["VIX"] > df["VIX_MA5"])

# -----------------------------
# Create position series
# -----------------------------
position = []
pos = 0  # 1 = long, 0 = flat

for i in range(len(df)):
    if pos == 0:  
        # currently flat → check entry
        if enter.iloc[i]:
            pos = 1
    else:
        # currently long → check exit
        if exit_.iloc[i]:
            pos = 0
    position.append(pos)

df["position"] = position

ValueError: Operands are not aligned. Do `left, right = left.align(right, axis=1, copy=False)` before operating.

In [ ]:
# -----------------------------
# Daily returns
# -----------------------------
df["UPRO_ret"] = df["UPRO"].pct_change().fillna(0)

# Strategy return = UPRO return * position
df["strategy_ret"] = df["UPRO_ret"] * df["position"]

# NAV calculation
df["NAV_strategy"] = (1 + df["strategy_ret"]).cumprod()
df["NAV_buyhold"] = (1 + df["UPRO_ret"]).cumprod()

In [ ]:
# -----------------------------
# Performance statistics
# -----------------------------
def annualized_return(nav):
    total_ret = nav.iloc[-1] / nav.iloc[0]
    N = len(nav)
    return total_ret ** (252 / N) - 1

def annualized_vol(returns):
    return returns.std() * np.sqrt(252)

def sharpe(ret, vol):
    return ret / vol if vol != 0 else np.nan

def max_drawdown(nav):
    roll_max = nav.cummax()
    dd = nav / roll_max - 1
    mdd = dd.min()
    end = dd.idxmin()
    start = nav.loc[:end].idxmax()
    return mdd, start, end

In [ ]:
# Metrics
ann_ret = annualized_return(df["NAV_strategy"])
ann_vol = annualized_vol(df["strategy_ret"])
sharpe_ratio = sharpe(ann_ret, ann_vol)
mdd, mdd_start, mdd_end = max_drawdown(df["NAV_strategy"])

ann_ret_bh = annualized_return(df["NAV_buyhold"])
ann_vol_bh = annualized_vol(df["UPRO_ret"])
sharpe_bh = sharpe(ann_ret_bh, ann_vol_bh)
mdd_bh, mdd_bh_start, mdd_bh_end = max_drawdown(df["NAV_buyhold"])

In [ ]:
# Output results
# -----------------------------
print("=== Strategy (VIX rule) ===")
print(f"Final NAV: {df['NAV_strategy'].iloc[-1]:.4f}")
print(f"Annualized return: {ann_ret*100:.2f}%")
print(f"Annualized vol: {ann_vol*100:.2f}%")
print(f"Sharpe (rf=0.0): {sharpe_ratio:.2f}")
print(f"Max Drawdown: {mdd*100:.2f}%  (from {mdd_start.date()} to {mdd_end.date()})")
print()

print("=== Buy & Hold UPRO ===")
print(f"Final NAV: {df['NAV_buyhold'].iloc[-1]:.4f}")
print(f"Annualized return: {ann_ret_bh*100:.2f}%")
print(f"Annualized vol: {ann_vol_bh*100:.2f}%")
print(f"Sharpe (rf=0.0): {sharpe_bh:.2f}")
print(f"Max Drawdown: {mdd_bh*100:.2f}%  (from {mdd_bh_start.date()} to {mdd_bh_end.date()})")
print()

print("Trades executed:", (df["position"].diff().abs().sum()))
print("Total days in market:", df["position"].sum())
print()

print("Recent data:")
print(df.tail(10))
